# EPySuite Demo

This notebook demonstrates the basic functionality of the EPySuite package.

## Basic Usage

First, let's import the necessary classes and create a runner instance. Then we can add a test case to the runner and run it for `Benzene`. By default EPI Suite™ will run STPWin to predict the removal of a chemical in a typical activated sludge-based sewage system using the BIOWIN model. The output will be a `Polars` dataframe with columns for each result. The snippet below shows the output for a few of the most relevant columns for the BIOWIN model.

In [19]:
from epysuite import EPySuiteConfig, EPySuiteRunner

# Initialize the runner with default configuration
runner = EPySuiteRunner()

# Get data for benzene
results = runner.get_data(
    cas_rn="71-43-2",
    smiles="c1ccccc1",
)

# Show select columns from the results
results['Chemical name', 'SMILES', 'Estimated Log Kow ', 'Measured Log Kow  ','STP Total Removal (%)']


Chemical name,SMILES,Estimated Log Kow,Measured Log Kow,STP Total Removal (%)
str,str,f64,f64,f64
"""71-43-2""","""c1ccccc1""",1.99,2.13,76.22


## Sewage Treatment Plant (STP) Batch Processing

The `EPySuite` package also supports batch processing of multiple chemicals with or without SMILES. In the following example, several chemicals without SMILES are processed in a single batch using EPI Suite™ to lookup the SMILES for each chemical. The EPI Suite™ STP model predicts the fate of a chemical in a conventional activated sludge treatment plant. The model considers three main removal processes:

1. **Biodegradation**: Microbial breakdown of the chemical
2. **Sludge Adsorption**: Attachment of the chemical to solid particles that settle out
3. **Volatilization**: Transfer of the chemical from water to air

These results are displayed below for the selected chemicals.

In [20]:
import pandas as pd

# List of chemicals to analyze
chemicals = [
    ("71-43-2", "Benzene"),
    ("67-64-1", "Acetone"),
    ("67-66-3", "Chloroform"),
    ("50-32-8", "Benzo[a]pyrene"),
    ("108-88-3", "Toluene")
]

# Initialize runner
runner = EPySuiteRunner()

# Get data for all chemicals and store results
all_results = []
for cas, name in chemicals:
    result = runner.get_data(cas_rn=cas)
    all_results.append((name, result))

# Create lists for plotting
chemicals = []
biodeg_values = []
sludge_values = []
vol_values = []
remaining_values = []

# Extract values for each chemical
for name, result in all_results:
    chemicals.append(name)
    biodeg_values.append(result.select('STP Biodegradation (%)').item())
    sludge_values.append(result.select('STP Sludge Adsorption (%)').item())
    vol_values.append(result.select('STP Volatilization (%)').item())
    total = result.select('STP Total Removal (%)').item()
    remaining_values.append(100 - total)

# Show table of results
pd.DataFrame({
    'Chemical': chemicals,
    'Biodegradation (%)': biodeg_values,
    'Sludge Adsorption (%)': sludge_values,
    'Volatilization (%)': vol_values,
    'Remaining (%)': remaining_values
})

,Chemical,Biodegradation (%),Sludge Adsorption (%),Volatilization (%),Remaining (%)
0,Benzene,21.66,0.97,53.59,23.78
1,Acetone,91.46,0.33,0.31,7.90
2,Chloroform,10.25,1.07,52.85,35.83
3,Benzo[a]pyrene,25.83,71.21,0.00,2.96
4,Toluene,76.27,1.36,17.34,5.04


## Additional Analyses and Data Visualization

Additional analysis can be performed after running the model. For example, the results can be displayed in a plot to compare the relative STP removal rates for each chemical.

In [23]:
import plotly.graph_objects as go

# Create stacked bar chart
fig = go.Figure()

# Add bars for each process
fig.add_trace(go.Bar(
    y=chemicals,
    x=biodeg_values,
    name='Biodegradation',
    orientation='h',
    marker_color='rgb(99, 110, 250)',
    text=[f"{x:.1f}%" for x in biodeg_values],
    textposition='inside'
))

fig.add_trace(go.Bar(
    y=chemicals,
    x=sludge_values,
    name='Sludge Adsorption',
    orientation='h',
    marker_color='rgb(239, 85, 59)',
    text=[f"{x:.1f}%" for x in sludge_values],
    textposition='inside'
))

fig.add_trace(go.Bar(
    y=chemicals,
    x=vol_values,
    name='Volatilization',
    orientation='h',
    marker_color='rgb(0, 204, 150)',
    text=[f"{x:.1f}%" for x in vol_values],
    textposition='inside'
))

fig.add_trace(go.Bar(
    y=chemicals,
    x=remaining_values,
    name='Remaining in System',
    orientation='h',
    marker_color='rgb(200, 200, 200)',
    text=[f"{x:.1f}%" for x in remaining_values],
    textposition='inside'
))

# Update layout
fig.update_layout(
    title='Comparison of Chemical Fate in Sewage Treatment Plants',
    barmode='stack',
    showlegend=True,
    xaxis_title='Percentage',
    width=900,
    height=400,
    xaxis=dict(range=[0, 100]),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.5,
        xanchor="center",
        x=0.5
    ),
    margin=dict(b=100)  # Add more bottom margin for legend
)

# Show the figure
fig.show()